# Supporting Microsoft’s GraphRAG: Part 3

To run this Jupyter Notebook, you can download the original `.ipynb` file from [msft_graphrag_3.ipynb](https://github.com/xuanleilin/tigergraphx/tree/main/docs/graphrag/msft_graphrag_3.ipynb).

---

## Retrieve the Graph

In [2]:
from tigergraphx import Graph, TigerGraphConnectionConfig
connection = TigerGraphConnectionConfig.ensure_config({
    "host": "http://127.0.0.1",
    "user_name": "tigergraph",
    "password": "tigergraph",
})
G = Graph.from_db("GraphRAG", connection)

---

## What’s Next?

- [API Reference](../../reference/features_overview): Dive deeper into TigerGraphX APIs.

---

Start transforming your GraphRAG workflows with the power of **TigerGraphX** today!